In [1]:
import pandas as pd
import numpy as np
from datetime import time

In [90]:
df = pd.read_csv("hourly_volume_data_eastern.csv").iloc[:,-1:]
df["cleaned_time"] = vol.apply(lambda x: x['Eastern_Time'][:-6], axis = 1)
df['cleaned_time'] = pd.to_datetime(vol['cleaned_time'])
df.head()

,Eastern_Time,cleaned_time
0,2024-01-02 09:30:00-05:00,2024-01-02 09:30:00
1,2024-01-02 10:30:00-05:00,2024-01-02 10:30:00
2,2024-01-02 11:30:00-05:00,2024-01-02 11:30:00
3,2024-01-02 12:30:00-05:00,2024-01-02 12:30:00
4,2024-01-02 13:30:00-05:00,2024-01-02 13:30:00


In [91]:

# Extract date only (for daily-level labeling)
df['date_only'] = df['cleaned_time'].dt.date
df.drop_duplicates('date_only', inplace = True)

# Extract day of the week
df['day_of_week'] = df['cleaned_time'].dt.day_name() # Get day name (Monday, Tuesday, etc.)

# Create a daily DataFrame (only one row per date)
df_daily = df[['date_only']].drop_duplicates().reset_index(drop=True)

# First and Last Trading Day of the Month
df_daily['first_trading_day'] = df_daily.groupby(pd.to_datetime(df_daily['date_only']).dt.to_period('M'))['date_only'].transform('min')
df_daily['last_trading_day'] = df_daily.groupby(pd.to_datetime(df_daily['date_only']).dt.to_period('M'))['date_only'].transform('max')

# Last Trading Day of the Quarter
df_daily['last_trading_day_of_quarter'] = df_daily.groupby(pd.to_datetime(df_daily['date_only']).dt.to_period('Q'))['date_only'].transform('max')

# Option Expiry Date (Third Friday of each month)
def get_option_expiry(d):
    first_day = d.replace(day=1)
    first_friday = first_day + pd.DateOffset(days=(4 - first_day.weekday() + 7) % 7)
    third_friday = first_friday + pd.DateOffset(weeks=2)
    return third_friday.date()

df_daily['third_friday'] = df_daily['date_only'].apply(lambda d: get_option_expiry(pd.to_datetime(d)))
df_daily['option_expiry'] = df_daily.groupby(pd.to_datetime(df_daily['date_only']).dt.to_period('M'))['third_friday'].transform('max')

# Convert to binary dummy features
df_daily['is_option_expiry'] = (df_daily['date_only'] == df_daily['option_expiry']).astype(int)
df_daily['is_first_trading_day'] = (df_daily['date_only'] == df_daily['first_trading_day']).astype(int)
df_daily['is_last_trading_day'] = (df_daily['date_only'] == df_daily['last_trading_day']).astype(int)
df_daily['is_last_trading_day_of_quarter'] = (df_daily['date_only'] == df_daily['last_trading_day_of_quarter']).astype(int)

# Dummify the day_of_week
day_of_week_dummies = pd.get_dummies(pd.to_datetime(df_daily['date_only']).dt.day_name(), dtype=int)
df_daily = pd.concat([df_daily, day_of_week_dummies], axis=1)

# Merge back to original hourly dataframe based on `date_only`
df = df.merge(df_daily, on='date_only', how='left')

# Drop unnecessary columns
df = df.drop(columns=['first_trading_day', 'last_trading_day', 'last_trading_day_of_quarter', 'third_friday', 'option_expiry'])

In [92]:
df.columns

Index(['Eastern_Time', 'cleaned_time', 'date_only', 'day_of_week',
       'is_option_expiry', 'is_first_trading_day', 'is_last_trading_day',
       'is_last_trading_day_of_quarter', 'Friday', 'Monday', 'Thursday',
       'Tuesday', 'Wednesday'],
      dtype='object')

In [93]:
df.to_csv("dates_feature.csv",index=False)